In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [2]:
browser = Browser('chrome', {'executable_path': 'chromedriver.exe'})

## NASA Mars News

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html')

In [4]:
content_titles = soup.find_all('div', class_ = 'content_title')
article_title = content_titles[1].text
article_title

"How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus "

In [5]:
teaser_body = soup.find('div', class_ = 'article_teaser_body').text
teaser_body

'Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.'

## JPL Mars Space Images - Featured Image

In [22]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [23]:
browser.click_link_by_partial_text('FULL')

In [24]:
browser.click_link_by_partial_text('more info')
html = browser.html
soup = bs(html,'html')

In [25]:
lede = soup.find('figure', class_='lede')
img = lede.find('a')['href']
img_url = f'https://jpl.nasa.gov{img}'
img_url

'https://jpl.nasa.gov/spaceimages/images/largesize/PIA19685_hires.jpg'

## Mars Weather

In [37]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
soup = bs(html, 'html')

In [39]:
divs = soup.find_all('div')
di

xmlns="http://www.w3.org/2000/svg"><g><path d="M23.643 4.937c-.835.37-1.732.62-2.675.733.962-.576 1.7-1.49 2.048-2.578-.9.534-1.897.922-2.958 1.13-.85-.904-2.06-1.47-3.4-1.47-2.572 0-4.658 2.086-4.658 4.66 0 .364.042.718.12 1.06-3.873-.195-7.304-2.05-9.602-4.868-.4.69-.63 1.49-.63 2.342 0 1.616.823 3.043 2.072 3.878-.764-.025-1.482-.234-2.11-.583v.06c0 2.257 1.605 4.14 3.737 4.568-.392.106-.803.162-1.227.162-.3 0-.593-.028-.877-.082.593 1.85 2.313 3.198 4.352 3.234-1.595 1.25-3.604 1.995-5.786 1.995-.376 0-.747-.022-1.112-.065 2.062 1.323 4.51 2.093 7.14 2.093 8.57 0 13.255-7.098 13.255-13.254 0-.2-.005-.402-.014-.602.91-.658 1.7-1.477 2.323-2.41z"></path></g></svg><span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"></span></div></a></h1></div><div class="css-1dbjc4n r-1oszu61 r-18u37iz r-16y2uox r-1wbh5a2 r-zg41ew"><div class="css-1dbjc4n r-1awozwy r-16y2uox r-1pi2tsx r-1777fci r-utggzx r-tvv088"><div aria-valuemax="1" aria-valuemin="0" class="css-1dbj